# 4. Run GSEA methods on DE output
#### Workflow Overview
* Run prerank on all contrasts using gene sets defined in second cell
* Run traditional GSEA using normalized counts with defined genesets
* Run single sample GSEA using normalized counts with defined genesets

#### Expected Input Files
- The notebook expects DESeq2 AnnData objects from the previous notebook.

#### Output Files
- Updated AnnData objects
    * Prerank results saved to uns['gsea_results_prerank']
    * Traditional GSEA results saved to uns['gsea_results_standard']
    * Single sample GSEA results saved to obsm as arrays with column keys in uns['ssgsea_colnames']

In [ ]:
from pathlib import Path 
import warnings

import pandas as pd
import numpy as np
from pybiomart import Dataset
import anndata as ad
import matplotlib.pyplot as plt

import gseapy as gp
from gseapy import Msigdb
import gseapy.parser as gmtparser

warnings.filterwarnings('ignore')

### 4.1 Configure Notebook

#### Required Variable Definitions:
Processing Parameters
- *NUM_CPUS*: Number of CPUs to use for parallel processing (default: 8)
- *GENE_SET_ANNOTATIONS*: Gene sets to use for GSEA. Must be switched to mouse gene_sets if mouse experiment. 
    - Available gene sets defined:
        * At bottom of page here: https://gseapy.readthedocs.io/en/latest/gseapy_tutorial.html via enrichr API.
        * Here: https://gseapy.readthedocs.io/en/latest/gseapy_example.html#Msigdb-API via Msigdb API.
- *GENE_PREFIX*: Gene prefix for ensembl IDs, must be switched to mouse gene_sets if mouse experiment. 
- *DB_VER*: Msigdb version str. 
File Paths
- *DATA_PATH*: Root directory path for the project (default: current working directory)
- *RESULTS_PATH*: Output directory for differential expression results (default: DATA_PATH/de_results)
- *DDS_GENE_FH*: Path to gene-level DESeq2 dataset object

In [ ]:
NUM_CPUS = 8

DATA_PATH = Path.cwd().parent

EXPERIMENT_ID = DATA_PATH.parts[-1]

RESULTS_PATH = DATA_PATH / 'de_results'

DDS_GENE_FH = RESULTS_PATH / f'{EXPERIMENT_ID}_dds_gene.h5_ad'

# Human gene prefix. 
GENE_PREFIX = "ENSG"

# # Mouse gene prefix. 
# GENE_PREFIX = "ENSMUS"

# Human set. 
GENE_SET_ANNOTATIONS = [
    'h.all', 
    'Reactome_2016',
    'KEGG_2016',
]

# # Mouse set.
# GENE_SET_ANNOTATIONS = [
#     'mh.all', 
#     'm2.cp.reactome',
#     'm2.cp.biocarta',
# ]

# Human msigdb. 
DBVER = '2023.1.Hs'

# # Mouse msigdb. 
# DBVER = '2023.1.Mm'

### 4.2 Download gene sets

In [ ]:
# Download all the GENE_SET_ANNOTATIONS. 

for ga in GENE_SET_ANNOTATIONS:
    try:
        gmtparser.download_library(ga)
    except:
        pass
        
# Pull MSigDB gene sets down.

msigdb_gene_sets = {}

msig = Msigdb()

for gs in GENE_SET_ANNOTATIONS:
    try:
        gmt = msig.get_gmt(category=gs, dbver=DBVER)
        msigdb_gene_sets[gs] = gmt
    except:
        pass

### 4.3 Define dataframes with ensembl ids

In [ ]:
# Load adata objects containing normed_counts for running traditional gsea.

dds_gene = ad.read_h5ad(DDS_GENE_FH)

# Copy DE dataframes to new dataframe that can be manipulated for gsea.

dds_gene.uns['stat_results_gsea'] = {k:v.copy() for k,v in dds_gene.uns['stat_results'].items()}

# Filter dataframes by gene_id, only keep ensembl gene ids.

for k in dds_gene.uns['stat_results_gsea'].keys():
    dds_gene.uns['stat_results_gsea'][k] = dds_gene.uns['stat_results_gsea'][k].loc[
        dds_gene.uns['stat_results_gsea'][k].index.str.startswith(gene_prefix)
    ]

### 4.4 Fetch gene_name/ensembl gene id mappings.

In [ ]:
# Fetch ensembl gene id - external gene name mappings from biomart.

species_mapping = 'hsapiens' if any(dds.var.index.str.startswith('ENSG')) else (
    'mmusculus' if any(dds.var.index.str.startswith('ENSMUSG')) else None
)
if not species_mapping:
    raise Exception('Must pick a valid species, adjust above if not human/mouse.')

dataset = Dataset(
    name=species_mapping,
    host='http://www.ensembl.org',
)

external_gene_mapping = dataset.query(
    attributes=['ensembl_gene_id', 'external_gene_name']
)

external_gene_mapping.rename({'Gene stable ID': 'gene_id', 'Gene name': 'gene_name'}, axis=1, inplace=True)

external_gene_mapping

In [ ]:
# Convert ensembl gene ids to stable ids by removing version. Merge external gene name.

for k in dds_gene.uns['stat_results_gsea'].keys():
    if 'gene_name' not in dds_gene.uns['stat_results_gsea'][k].columns:
        dds_gene.uns['stat_results_gsea'][k].index = dds_gene.uns['stat_results_gsea'][k].index.str.split('.').str[0]

        dds_gene.uns['stat_results_gsea'][k] = dds_gene.uns['stat_results_gsea'][k].merge(
            external_gene_mapping,
            left_index=True,
            right_on='gene_id',
        )

In [ ]:
# Drop rows where there isn't a defined gene_name.

for k in dds_gene.uns['stat_results_gsea'].keys():
    dds_gene.uns['stat_results_gsea'][k] = dds_gene.uns['stat_results_gsea'][k].loc[
        ~dds_gene.uns['stat_results_gsea'][k]['gene_name'].isnull()
    ]

### 4.5 Run GSEA prerank with Wald statistic.

In [ ]:
# Run GSEA prerank on Wald statistic ranks in gsea DE dataframes using gene sets of choice.

dds_gene.uns['gsea_results_prerank'] = {}

for ga in GENE_SET_ANNOTATIONS:

    for k in dds_gene.uns['stat_results_gsea'].keys():

        gs_res = gp.prerank(
            rnk=dds_gene.uns['stat_results_gsea'][k][['gene_name', 'stat']],
            gene_sets=msigdb_gene_sets.get(ga, ga)
            threads=NUM_CPUS,
            min_size=5, 
            max_size=1000,
            permutation_num=1000,
            outdir=None,
            seed=42,
            verbose=True,
        )
        
        dds_gene.uns['gsea_results_prerank']['%s_%s' % (k,ga)] = gs_res.res2d.astype(str, copy=True).copy()

        print('%s_%s' % (k, ga))
        display(gs_res.res2d.head(10))

        terms = gs_res.res2d.Term
        axs = gs_res.plot(terms=terms[0:20])
        axs.suptitle('%s_%s' % (k,ga), y=0.0)

### 4.6 Run traditional GSEA on normalized counts.

In [ ]:
# Create dataframes and class vectors for running GSEA on normed_counts. 

gene_df = pd.DataFrame(
    dds_gene.layers['normed_counts'].T, 
    columns=dds_gene.obs.index, 
    index=dds_gene.var.index,
)

# Filter dataframes by gene_id, only keep ensembl gene ids.
gene_df = gene_df.loc[gene_df.index.str.startswith(GENE_PREFIX)]

if any('.' in i for i in gene_df.index ):
    
    gene_df.index = gene_df.index.str.split('.').str[0]

    gene_df = gene_df.merge(external_gene_mapping, left_index=True, right_on='gene_id')

    gene_df = gene_df.loc[~gene_df['gene_name'].isnull()]

    gene_df.set_index('gene_name', inplace=True)
    
dds_gene.uns['gsea_gene_df'] = gene_df.copy()


In [ ]:
# Run GSEA on normed_counts for each contrast specied in dds object.

dds_gene.uns['gsea_results_standard'] = {}

for ga in GENE_SET_ANNOTATIONS:

    for k, v in dds_gene.uns['contrasts'].items():

        smallest_condition_size = dds_gene.obs[v[0]].value_counts()[-1]

        if smallest_condition_size < 3:
            continue

        # Samples need to be ordered in order for the enrichment sign to be in the correct direction.
        # Samples need to be ordered comp level, then ref level.
        
        ref_samples_in_contrast = dds_gene.obs[dds_gene.obs[v[0]] == v[2]].index
        comp_samples_in_contrast = dds_gene.obs[dds_gene.obs[v[0]] == v[1]].index

        conditions = dds_gene.obs.loc[comp_samples_in_contrast.append(ref_samples_in_contrast), v[0]]

        gs_res = gp.gsea(
            data=dds_gene.uns['gsea_gene_df'].loc[:,comp_samples_in_contrast.append(ref_samples_in_contrast)],
            gene_sets=msigdb_gene_sets.get(ga, ga),
            cls=conditions,
            permutation_type='phenotype' if len(conditions) > 15 else 'gene_set',
            permutation_num=1000,
            outdir=None,
            method='signal_to_noise',
            min_size=5,
            max_size=1000,
            threads=NUM_CPUS, 
            seed= 42,
            verbose=True,
        )
        
        dds_gene.uns['gsea_results_standard']['%s_%s' % (k, ga)] = gs_res.res2d.astype(str).copy()

        print('%s_%s' % (k, ga))
        print(gs_res.res2d.head(10).to_string())

        terms = gs_res.res2d.Term
        axs = gs_res.plot(terms=terms[0:20])
        axs.suptitle(k,y=0.0)

### 4.7 Run single-sample GSEA. Save pathway enrichment- and normalized enrichment- scores to anndata.

In [ ]:
# Run all samples through ssgsea, create matrix of ssgsea output in dds.obsm storing ssgsea NES and ES.
# There's an issue saving pandas DataFrames with large headers via hdf5, so saving ssgsea results in two
# arrays in obsm, and then saving header to separate arrays in uns.

dds_gene.uns['ssgsea_colnames'] = {}

for ga in GENE_SET_ANNOTATIONS:

    if '.all' in ga:
        gmt = msigdb_gene_sets[ga]
    else:
        gmt = gmtparser.get_library(ga)

    dds_gene.obsm['%s_ssgsea_es' % ga] = np.zeros((dds_gene.obs.shape[0], len(gmt)))

    dds_gene.obsm['%s_ssgsea_nes' % ga] = np.zeros((dds_gene.obs.shape[0], len(gmt))) 

    dds_gene.uns['ssgsea_colnames'][ga] = np.array([c for c in gmt.keys()])
            
    for i,s in enumerate(dds_gene.obs.index):
        
        ss = gp.ssgsea(
            data=dds_gene.uns['gsea_gene_df'].loc[:,s].rename(None),
            gene_sets=msigdb_gene_sets.get(ga, ga),
            outdir=None,
            sample_norm_method='rank', 
            no_plot=True,
            verbose=True,
            min_size=5, 
            max_size=1000,
        )

        ss.res2d.set_index('Term', inplace=True)

        for c in ss.res2d.index:

            c_index = np.where(dds_gene.uns['ssgsea_colnames'][ga] == c)[0][0]

            dds_gene.obsm['%s_ssgsea_es' % ga][i,c_index] = ss.res2d.loc[c,'ES']
            dds_gene.obsm['%s_ssgsea_nes' % ga][i,c_index] = ss.res2d.loc[c,'NES']
            

In [ ]:
# Save output of gsea analyses.

dds_gene.write(DDS_GENE_FH)